The two previous notebooks looked at over all p values and alpha diversity for a single category. Now, we'll look at beta diversity, which lets us compare community structure.

In [1]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats
import seaborn as sn
import skbio

import americangut.diversity_analysis as div
import americangut.notebook_environment as agenv

from americangut.ag_data import AgData
from americangut.ag_dictionary import ag_dictionary

/Users/jwdebelius/anaconda/envs/americangut/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [2]:
from matplotlib import rcParams

% matplotlib inline

# Formats the axes using seabron so they will be white, and have ticks
# on the bottom of the axes.
sn.set_style('ticks', {'axes.facecolor': 'none'})

# Sets up plotting parameters so that the default setting is use to Helvetica
# in plots
rcParams['font.family'] = 'sans-serif'
rcParams['font.sans-serif'] = ['Helvetica', 'Arial']
rcParams['text.usetex'] = True

Next, let's select the data set and rarefaction depth we wish to use.

In [ ]:
bodysite = 'fecal'
sequence_trim = '100nt'
rarefaction_depth = '10k'

use_subset = True
use_one_sample = True

Now, let's read the files assoicated with the data and load the data dictionary entry for the group.

In [ ]:
fecal_data = AgData(bodysite=bodysite, 
                    trim=sequence_trim, 
                    depth=rarefaction_depth, 
                    sub_participants=use_subset, 
                    one_sample=use_one_sample)

fecal_data.drop_alpha_outliers()
fecal_data.drop_bmi_outliers()

Let's make a directory where we can save any figures we need to save.

In [ ]:
save_dir = agenv.check_save_dir(fecal_data.data_set)
test_dir = os.path.join(save_dir, 'beta/dist/')
fig_dir = os.path.join(save_dir, 'beta/images/')

if not os.path.exists(fig_dir):
    os.makedirs(fig_dir)
if not os.path.exists(test_dir):
    os.makedirs(test_dir)

Let's pick the category to interogate.

In [ ]:
group_name = 'TYPES_OF_PLANTS'
metric = 'unweighted_unifrac'

We're going to clean up the data. 

In [ ]:
question = ag_dictionary(group_name)
fecal_data.clean_group(question)
fecal_data.filter_by_question(question)

Now that we have the data loaded, let's use a post-hoc test to evaluate comparisons between groups. For now, we're going to use the QIIME script, [`make_distance_boxplots.py`](http://qiime.org/scripts/make_distance_boxplots.html).

In [ ]:
if not os.path.exists(os.path.join(test_dir, '%s_Stats.txt' % question.name)):
    map_fp = fecal_data.map_fp
    dm_fp = fecal_data.beta_fp[metric]
    !make_distance_boxplots.py -o $test_dir -m $map_fp -d $dm_fp -f $group_name -n 999

In [ ]:
distance_summary = pd.read_csv(os.path.join(test_dir, '%s_Stats.txt') % question.name,
                               sep='\t',
                               skiprows=7)

We can use the generated data to make distance plots to compare the groups. The plots are not necessarily a standard representation. Here, we're comparing the within-group distance, given by the 

In [ ]:
beta_ax = div.beta_diversity_bars(fecal_data.beta[metric],
                                  fecal_data.map_, 
                                  question.name, 
                                  order=question.order, 
                                  num_iter=999,
                                  colormap='Greens',
                                  p_table=distance_summary,
                                  p_tab_col='Nonparametric p-value',
                                  ylim=[0.5, 1.0],
                                  ylabel='Unweighted UniFrac Distance',
                                  xlabel=question.clean_name,
                                  ref_less=True
                                 )
beta_ax.set_position((0.2, 0.2, len(question.order) * 0.15, 0.75))
beta_ax.figure.savefig(os.path.join(fig_dir, '%s_bars.pdf' % question.name), dpi=300)